# Import, Init

In [90]:
import os, sys
import pandas as pd
import seaborn as sns

sys.path.insert(0, "./classes/")
import cls_nnetwork

# reloading functions without runtime.restart
import importlib
importlib.reload(cls_nnetwork)

<module 'cls_nnetwork' from './classes/cls_nnetwork.py'>

In [94]:
# cut -- train test data with trimmed columns, so that it's more easy to train

nn_manager_cut = cls_nnetwork.NeuralkManager(dir_path='../data/csv_to_train/train_test_cut')

>>> train-test inited:  {'X_test_cut': True, 'X_train_cut': True, 'y_test_cut': True, 'y_train_cut': True}


In [95]:
nn_manager_cut.X_train.shape, nn_manager.X_test.shape

((3182, 12), (795, 12))

In [96]:
nn_manager_cut.X_train[:5]

,Open,High,Low,Vol,AVBLS,NTRAT,TOUTV,TRFEE,BTC_MINED_PDAY,VOL_CHANGE_PDAY,MWNUS_CH_PDAY,NTRAT_CH_PDAY
Date,,,,,,,,,,,,
2019-04-03,4859.3,5278.4,4814.0,13830000.0,1.248330,397961949.0,1.306792e+06,142.102988,425.0,-12780000.0,52351.0,360333.0
2019-04-02,4145.1,4899.6,4143.5,14770000.0,1.215507,397560778.0,3.429199e+06,114.489789,1512.5,-940000.0,61873.0,401171.0
2019-04-01,4102.3,4159.1,4076.8,3220000.0,1.205358,397191600.0,1.744093e+06,61.846737,2162.5,11550000.0,87465.0,369178.0
2019-03-31,4111.8,4121.9,4082.2,2430000.0,1.173566,396808790.0,9.168389e+05,44.614255,1425.0,790000.0,34688.0,382810.0
2019-03-30,4103.7,4138.1,4057.1,2550000.0,1.204920,396430531.0,9.228557e+05,54.673729,2050.0,-120000.0,64109.0,378259.0


# Model Creation

In [ ]:
nn_manager.`